In [0]:
# -*- coding: utf-8 -*-
"""ner_test.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1YxFNXg4DnYQxpEP06kFBV3WVlDQe2_hq
"""

import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [2]:
!wget https://media.githubusercontent.com/media/KolosDan/nerbert/master/data/ner_dataset.csv?token=AMN6MNDOEBIOYT56PGPSALK6HOZAO
# importing data

ner_dataset = '/content/ner_dataset.csv?token=AMN6MNDOEBIOYT56PGPSALK6HOZAO'

df = pd.read_csv(ner_dataset, encoding="latin1").fillna(method="ffill")
data = []

# formating data in batches
for group in df.groupby("Sentence #"):
    s = group[1]
    data.append(list(zip(s["Word"].values.tolist(),
    s["POS"].values.tolist(),
    s["Tag"].values.tolist())))

--2020-02-06 05:31:26--  https://media.githubusercontent.com/media/KolosDan/nerbert/master/data/ner_dataset.csv?token=AMN6MNDOEBIOYT56PGPSALK6HOZAO
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15208151 (15M) [text/plain]
Saving to: ‘ner_dataset.csv?token=AMN6MNDOEBIOYT56PGPSALK6HOZAO’

ner_dataset.csv?tok 100%[===================>]  14.50M  --.-KB/s    in 0.1s    

2020-02-06 05:31:27 (117 MB/s) - ‘ner_dataset.csv?token=AMN6MNDOEBIOYT56PGPSALK6HOZAO’ saved [15208151/15208151]



In [0]:
sentences = [" ".join([s[0] for s in sent]) for sent in data]
labels = [[s[2] for s in sent] for sent in data]
tags_vals = list(set(df["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}

In [4]:
!pip install pytorch-pretrained-bert

import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

     |████████████████████████████████| 133kB 3.5MB/s 


Using TensorFlow backend.


In [0]:
# defining max sentence len and input batch size
MAX_LEN = 75
bs = 32

In [0]:
# choosing device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [7]:
# loading BertTokenizer (pre-trained) and tokenizing sentences
tokenizer = BertTokenizer.from_pretrained('bert-large-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

100%|██████████| 213450/213450 [00:00<00:00, 2479413.84B/s]


In [0]:
# converting string tokens and tags to int ids and padding batches
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [0]:
# creating attention masks to avoid attention on padded sequences
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [0]:
# splitting data and converting to torch.Tensor type
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [0]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [0]:
# defining loader and sampler for training and validation
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

In [0]:
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [15]:
model = BertForTokenClassification.from_pretrained("bert-large-cased", num_labels=len(tag2idx))

model.cuda()

100%|██████████| 1242874899/1242874899 [00:21<00:00, 58275327.94B/s]


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fa

In [0]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [17]:
!pip install seqeval

from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

epochs = 5
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

pickle.dump(model.to (torch.device("cpu")), open("/content/nerbert_cpu.pkl", "wb"))

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=163f0182eb4f7dd9f02409d8af5a8080a9fbd79dca8fedaaddf742fc9ed4c256
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.254647870674409
Validation loss: 0.1472508489092191
Validation Accuracy: 0.921585714285714


Epoch:  20%|██        | 1/5 [39:53<2:39:32, 2393.14s/it]

F1-Score: 0.40393916728995266
Train loss: 0.13524825311230676
Validation loss: 0.12299616319437821
Validation Accuracy: 0.9231630952380954


...

Epoch:  80%|████████  | 4/5 [2:39:51<39:57, 2397.33s/it]  

F1-Score: 0.457841829931275
Train loss: 0.0729547574262848
Validation loss: 0.11700600183258454
Validation Accuracy: 0.9253408730158731


Epoch: 100%|██████████| 5/5 [3:19:50<00:00, 2397.72s/it]

F1-Score: 0.4556263482666934


Validation loss: 0.11700600183258454
Validation Accuracy: 0.9253408730158731
Validation F1-Score: 0.4556263482666934
